# Import necessary libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import concatenate, Input,BatchNormalization,Conv2D,Dropout,MaxPooling2D,LSTM,GRU,RNN,SimpleRNN
from tensorflow.keras.layers import Dense,Flatten,Reshape,TimeDistributed
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import regularizers
from tensorflow import keras
from tensorflow.keras import backend as K 
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import random
import numpy as np
import os
import shutil

# Input data

In [4]:

daily_train_path = 'E:/DeepTests/Data/Train/daily/'
daily_train_files = os.listdir(daily_train_path)
daily_test_path = 'E:/DeepTests/Data/Test/daily/'
daily_test_files = os.listdir(daily_test_path)


hourly_train_path = 'E:/DeepTests/Data/Train/hourly/6Hourly/'
houly_train_files = os.listdir(hourly_train_path)
hourly_test_path = 'E:/DeepTests/Data/Test/hourly/6Hourly/'
houly_test_files = os.listdir(hourly_test_path)

constant_train_path = 'E:/DeepTests/Data/Train/constant/'
constant_train_files = os.listdir(constant_train_path)
constant_test_path = 'E:/DeepTests/Data/Test/constant/'
constant_test_files = os.listdir(constant_test_path)

label_train_path = 'E:/DeepTests/Data/Train/label/'
label_train_files = os.listdir(label_train_path)
label_test_path = 'E:/DeepTests/Data/Test/label/'
label_test_files = os.listdir(label_test_path)

print('Test Len:',len(daily_test_files))
print('Train Len:',len(daily_train_files))

Test Len: 302
Train Len: 2522


# Matching the input data with labels

In [5]:
image_label_map = {
        "Train ({}).npy".format(i+1): "Train ({}).npy".format(i+1)
        for i in range(len(daily_train_files))}

image_label_map_val = {
        "Test ({}).npy".format(i+1): "Test ({}).npy".format(i+1)
        for i in range(len(daily_test_files))}

# Data Generator

In [6]:
btch=1

class DataGenerator(tf.keras.utils.Sequence):
  def __init__(self, list_examples, batch_size=btch, dim=(2,400,350),shuffle=False):
    # Constructor of the data generator.
    self.dim = dim
    self.batch_size = batch_size
    self.list_examples = list_examples
    self.shuffle = shuffle
    self.on_epoch_end()

  def __len__(self):
    # Denotes the number of batches per epoch
    return int(np.floor(len(self.list_examples) / self.batch_size))

  def __getitem__(self, index):
    # Generate one batch of data
    indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

    # Find list of IDs
    list_IDs_temp = [self.list_examples[k] for k in indexes]

    # Generate data
    X, y = self.__data_generation(list_IDs_temp)

    return X, y

  def on_epoch_end(self):
    # This function is called at the end of each epoch.
    self.indexes = np.arange(len(self.list_examples))
    if self.shuffle == True:
      np.random.shuffle(self.indexes)

  def __data_generation(self, list_IDs_temp):
    # Load individual numpy arrays and aggregate them to a batch.
    
    X1 = np.empty([self.batch_size, self.dim[0], self.dim[1],self.dim[2],8],dtype='float32') #Hourly data
    X2 = np.empty([self.batch_size, self.dim[0], self.dim[1],self.dim[2],3],dtype='float32') #Daily 
    X3 = np.empty([self.batch_size, self.dim[1],self.dim[2],5],dtype='float32') #Constant data
    # y is a one-hot encoded vector.
    y = np.empty([self.batch_size,self.dim[1],self.dim[2],1], dtype=np.float32)

    # Generate data.

    c=0
    for i in list_IDs_temp:
        
        x_daily_file_path = os.path.join(daily_train_path, i)
        y_file_path = os.path.join(label_train_path, image_label_map.get(i))
        
        x_constant_file_path = os.path.join(constant_train_path, i)
        x_houly_file_path = os.path.join(hourly_train_path, i)
        # Load sample 
        X1[c, :,:,:,:] = np.load(x_houly_file_path)
        X2[c, :,:,:,:] = np.load(x_daily_file_path)
        X3[c,:,:,:] = np.load(x_constant_file_path)
        # Load labels     
       
        y[c,:,:,:] = np.load(y_file_path)
        
        c=c+1
        
    return (X1,X2,X3), y



class ValDataGenerator(tf.keras.utils.Sequence):
  def __init__(self, list_examples, batch_size=btch, dim=(2,400,350),shuffle=False):
    # Constructor of the data generator.
    self.dim = dim
    self.batch_size = batch_size
    self.list_examples = list_examples
    self.shuffle = shuffle
    self.on_epoch_end()

  def __len__(self):
    # Denotes the number of batches per epoch
    return int(np.floor(len(self.list_examples) / self.batch_size))

  def __getitem__(self, index):
    # Generate one batch of data
    indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

    # Find list of IDs
    list_IDs_temp = [self.list_examples[k] for k in indexes]

    # Generate data
    X, y = self.__data_generation(list_IDs_temp)

    return X, y

  def on_epoch_end(self):
    # This function is called at the end of each epoch.
    self.indexes = np.arange(len(self.list_examples))
    if self.shuffle == True:
      np.random.shuffle(self.indexes)

  def __data_generation(self, list_IDs_temp):
    # Load individual numpy arrays and aggregate them to a batch.
    
    X1 = np.empty([self.batch_size, self.dim[0], self.dim[1],self.dim[2],8],dtype='float32') #Hourly Data
    X2 = np.empty([self.batch_size, self.dim[0], self.dim[1],self.dim[2],3],dtype='float32') #Daily Data
    X3 = np.empty([self.batch_size, self.dim[1],self.dim[2],5],dtype='float32') #Constant Data
    
    # y is a one-hot encoded vector.
    y = np.empty([self.batch_size,self.dim[1],self.dim[2],1], dtype=np.float32)

    # Generate data.

    c=0
    for i in list_IDs_temp:
        
        x_daily_file_path = os.path.join(daily_test_path, i)
        y_file_path = os.path.join(label_test_path, image_label_map_val.get(i))
        
        x_constant_file_path = os.path.join(constant_test_path, i)
        x_houly_file_path = os.path.join(hourly_test_path, i)
        # Load sample 
        X1[c, :,:,:,:] = np.load(x_houly_file_path)
        X2[c, :,:,:,:] = np.load(x_daily_file_path)
        X3[c,:,:,:] = np.load(x_constant_file_path)
        # Load labels     
       
        y[c,:,:,:] = np.load(y_file_path)
        
        c=c+1
        
    return (X1,X2,X3), y

In [7]:
training_generator = DataGenerator(daily_train_files)
validation_generator = ValDataGenerator(daily_test_files)

In [8]:
a=next(iter(training_generator))[0]# Input. 'a' is a tuple with the lenght og 3. Format: (Hourly, Daily,Constant)
b=next(iter(training_generator))[1] #Label
print('Label: ',b.shape) # (Batch size, n_rows,n_cols,n_channels)
print('Hourly data: ',a[0].shape) # (Batch size, n_rows,n_cols,n_channels)
print('Daily data: ',a[1].shape) # (Batch size, n_rows,n_cols,n_channels)
print('Constant data: ',a[2].shape) # (Batch size, n_rows,n_cols,n_channels)


Label:  (1, 400, 350, 1)
Hourly data:  (1, 2, 400, 350, 8)
Daily data:  (1, 2, 400, 350, 3)
Constant data:  (1, 400, 350, 5)


# Loss functions

In [9]:
def dice_coef(y_true, y_pred, smooth=1):
    """
    Dice = (2*|X & Y|)/ (|X|+ |Y|)
         =  2*sum(|A*B|)/(sum(A^2)+sum(B^2))
    ref: https://arxiv.org/pdf/1606.04797v1.pdf
    """
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    return (2. * intersection + smooth) / (K.sum(K.square(y_true),-1) + K.sum(K.square(y_pred),-1) + smooth)

def dice_coef_loss(y_true, y_pred):
    return (1-dice_coef(y_true, y_pred))

smooth=1
def tversky(y_true, y_pred):
    y_true_pos = K.flatten(y_true)
    y_pred_pos = K.flatten(y_pred)
    true_pos = K.sum(y_true_pos * y_pred_pos)
    false_neg = K.sum(y_true_pos * (1-y_pred_pos))
    false_pos = K.sum((1-y_true_pos)*y_pred_pos)
    alpha = 0.7
    return (true_pos + smooth)/(true_pos + alpha*false_neg + (1-alpha)*false_pos + smooth)

def tversky_loss(y_true, y_pred):
    return 1 - tversky(y_true,y_pred)

def iou(y_true, y_pred):
    intersection = K.sum(y_true * y_pred)
    sum_ = K.sum(y_true + y_pred)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return jac

def iou_loss(y_true, y_pred):
    return 1-iou(y_true, y_pred)

# Hourly block

In [10]:
def hourly_model(input_1):
    #Hourly Model
    
    x2 = TimeDistributed(Conv2D(16, (3,3), activation='relu',padding='same'))(input_1)  
    x2 = TimeDistributed(MaxPooling2D(pool_size=(2, 2))) (x2) 
    x2 = TimeDistributed(MaxPooling2D(pool_size=(2, 2))) (x2) 
    x2 = TimeDistributed(Conv2D(32, (3,3), activation='relu',padding='same')) (x2)
    x2 = TimeDistributed(MaxPooling2D(pool_size=(2, 2))) (x2) 
    x2 = TimeDistributed(MaxPooling2D(pool_size=(2, 2))) (x2) 
    
    x3 = TimeDistributed(Conv2D(32, (3,3),dilation_rate=3, activation='relu',padding='same')) (x2)
    x4 = TimeDistributed(Conv2D(32, (3,3),dilation_rate=6, activation='relu',padding='same')) (x2)
    x5 = TimeDistributed(Conv2D(32, (3,3),dilation_rate=12, activation='relu',padding='same')) (x2)
    x6 = TimeDistributed(Conv2D(32, (3,3),dilation_rate=1, activation='relu',padding='same')) (x2)
    
    x6=concatenate([x3,x4,x5,x6])
    
    x2 = TimeDistributed(Conv2D(1, (1,1), activation='relu',padding='same')) (x6)
    x2 = TimeDistributed(MaxPooling2D(pool_size=(2, 2))) (x2) 
    x2 = TimeDistributed(Conv2D(16, (3,3), activation='relu',padding='same')) (x2)
    x2 = TimeDistributed(MaxPooling2D(pool_size=(2, 2))) (x2) 
    x2 = TimeDistributed(Conv2D(32, (3,3), activation='relu',padding='same')) (x2)
    x2 = TimeDistributed(MaxPooling2D(pool_size=(2, 2))) (x2) 
    
    x2=TimeDistributed(BatchNormalization())(x2)
    
    x2 = TimeDistributed(Flatten()) (x2)
    
    return x2

# Hourly block output

In [11]:
input_1=Input((2,400,350,8))
out1=hourly_model(input_1)
out1.shape

TensorShape([None, 2, 192])

# Daily Block

In [12]:
def daily_model(input_2):
    #Daily Model
    
    x2 = TimeDistributed(Conv2D(16, (3,3), activation='relu',padding='same'))(input_2)  
    x2 = TimeDistributed(MaxPooling2D(pool_size=(2, 2))) (x2) 
    x2 = TimeDistributed(MaxPooling2D(pool_size=(2, 2))) (x2) 
    x2 = TimeDistributed(Conv2D(32, (3,3), activation='relu',padding='same')) (x2)
    x2 = TimeDistributed(MaxPooling2D(pool_size=(2, 2))) (x2) 
    x2 = TimeDistributed(MaxPooling2D(pool_size=(2, 2))) (x2) 
    
    x3 = TimeDistributed(Conv2D(32, (3,3),dilation_rate=3, activation='relu',padding='same')) (x2)
    x4 = TimeDistributed(Conv2D(32, (3,3),dilation_rate=6, activation='relu',padding='same')) (x2)
    x5 = TimeDistributed(Conv2D(32, (3,3),dilation_rate=12, activation='relu',padding='same')) (x2)
    x6 = TimeDistributed(Conv2D(32, (3,3),dilation_rate=1, activation='relu',padding='same')) (x2)
    
    x6=concatenate([x3,x4,x5,x6])
    x2 = TimeDistributed(Conv2D(1, (1,1), activation='relu',padding='same')) (x6)
    x2 = TimeDistributed(MaxPooling2D(pool_size=(2, 2))) (x2) 
    x2 = TimeDistributed(Conv2D(16, (3,3), activation='relu',padding='same')) (x2)
    x2 = TimeDistributed(MaxPooling2D(pool_size=(2, 2))) (x2) 
    x2 = TimeDistributed(Conv2D(32, (3,3), activation='relu',padding='same')) (x2)
    x2 = TimeDistributed(MaxPooling2D(pool_size=(2, 2))) (x2) 
    
    x2=TimeDistributed(BatchNormalization())(x2)
    
    x2 = TimeDistributed(Flatten()) (x2)
    
    return x2

# Daily block output

In [13]:
input_2=Input((2,400,350,3))
out2=daily_model(input_2)
out2.shape

TensorShape([None, 2, 192])

# Constant Block

In [14]:
def constant_model(input_3):
    
    x2 = Conv2D(16, (3,3), activation='relu',padding='same')(input_3)   
    x2=MaxPooling2D(pool_size=(2, 2)) (x2) 
    x2=MaxPooling2D(pool_size=(2, 2)) (x2) 
    x2 = Conv2D(32, (3,3), activation='relu',padding='same') (x2)
    x2=MaxPooling2D(pool_size=(2, 2)) (x2) 
    x2=MaxPooling2D(pool_size=(2, 2)) (x2) 
    
    x3 = Conv2D(32, (3,3),dilation_rate=3, activation='relu',padding='same') (x2)
    x4 = Conv2D(32, (3,3),dilation_rate=6, activation='relu',padding='same') (x2)
    x5 = Conv2D(32, (3,3),dilation_rate=12, activation='relu',padding='same') (x2)
    x6 = Conv2D(32, (3,3),dilation_rate=1, activation='relu',padding='same') (x2)
    
    x6=concatenate([x3,x4,x5,x6])
    x2 = Conv2D(1, (1,1), activation='relu',padding='same') (x2)
    x2 = MaxPooling2D(pool_size=(2, 2)) (x2) 
    x2 = Conv2D(16, (3,3), activation='relu',padding='same') (x2)
    x2 = MaxPooling2D(pool_size=(2, 2)) (x2) 
    x2 = Conv2D(32, (3,3), activation='relu',padding='same') (x2)
    x2 = MaxPooling2D(pool_size=(2, 2)) (x2) 
    
    x2=BatchNormalization()(x2)
    
    x2 = Flatten() (x2)
    
    return x2


# Constant block output

In [15]:
input_3=Input((400,350,5))
out3=constant_model(input_3)
out3.shape

TensorShape([None, 192])

# Combination (FirePred model)

In [16]:
def Model_(input_shape1,input_shape2,input_shape3):
    
    #Get Inputs
    input_1=Input(input_shape1)
    input_2=Input(input_shape2)
    input_3=Input(input_shape3)
    
    # Hourly features
    out1=hourly_model(input_1)
    # Daily Features
    out2=daily_model(input_2)
    #Constatnt Features
    out3=constant_model(input_3)
    
    #################################################
    out1=SimpleRNN(64,return_sequences=True,activation='relu') (out1)
    out1=SimpleRNN(32,return_sequences=False,activation='relu') (out1)
    
    out2=SimpleRNN(64,return_sequences=True,activation='relu') (out2)
    out2=SimpleRNN(32,return_sequences=False,activation='relu') (out2)
    
    out3=Dense(64,activation='relu') (out1)
    out3=Dense(32,activation='relu') (out1)
    
    x = concatenate([out1,out2,out3])
    
    x = Dense(128,activation='relu') (x)
    x = Dropout(0.4)(x)
    x = Dense(140000, activation='sigmoid') (x)  
    output = tf.keras.layers.Reshape((400,350,1)) (x)
    return Model(inputs=(input_1,input_2,input_3),outputs=output)
    ################################################
    
    
inp1=(2,400,350,8)
inp2=(2,400,350,3)
inp3=(400,350,5)
    
model=Model_(inp1,inp2,inp3)
model.summary()    

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 2, 400, 350, 0                                            
__________________________________________________________________________________________________
time_distributed_36 (TimeDistri (None, 2, 400, 350,  1168        input_4[0][0]                    
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, 2, 400, 350, 0                                            
__________________________________________________________________________________________________
time_distributed_37 (TimeDistri (None, 2, 200, 175,  0           time_distributed_36[0][0]        
______________________________________________________________________________________________

In [44]:
learning_rate = 0.0005
decay_rate = learning_rate / 200

loss_binary_=tf.keras.losses.BinaryCrossentropy()


optimizer = tf.keras.optimizers.Adam(lr=learning_rate,beta_1=0.9, beta_2=0.999, epsilon=None, decay=decay_rate, amsgrad=False)

model.compile(loss=loss_binary_, optimizer=optimizer, metrics=[iou]) 
early_stopping = keras.callbacks.EarlyStopping(monitor="val_jac_distance", patience=25)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=25)

model_path_save='F:/Researches/Forest Fire Researche/temporal/Model2.h5'
chek_point = ModelCheckpoint(model_path_save, verbose=1, save_best_only=True)    
callbacks=[chek_point,early_stopping,reduce_lr]

In [ ]:
hst = model.fit(training_generator,epochs=200,validation_data=validation_generator,callbacks=callbacks)